In [1]:
import sys
#sys.path.insert(0,'/content/drive/MyDrive/Colab Notebooks') #cambiar ruta segun donde este el archivo 'cuentitas.py' - es probable que deba dar permiso a drive desde el panel izquierdo
import cuentitas
import entornomaximo
from integracion import integracionGauss as ig
 
from numpy import pi, sqrt, sin, cos, exp, log10, array, real, conj
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from scipy import interpolate
import seaborn as sns
import pandas as pd
import glob
import theano.tensor as TT
 
import pymc3 as pm
#from google.colab import files
sns.set() #seaborn como default

In [2]:
#modelo SSA

# Amplitudes orden uno con una sola interface
#from ssa_enero_2021 import X1

# Amplitudes orden uno sup rugosa superior
#from ssa_enero_2021 import X1_u # u es upper

# Amplitudes orden uno sup rugosa inferior
#from ssa_enero_2021 import X1_b # b es bottom

#from ssa_enero_2021 import kx, ky, W

# TODAS LAS X1 son matrices:
# X1_algo[0,0] = VV

m = 51
X,Y,Wt = ig(m-1)


In [3]:
def S0VV_Fran_completo(k0,thi,phi,th,phs,ep1,ep2,d,s1,l1,s2,l2):
    
    k1 = k0*(sin(th)*cos(phs)-sin(thi)*cos(phi))
    k2 = k0*(sin(th)*sin(phs)-sin(thi)*sin(phi))
    
    return 4*np.pi*k0**2*cos(th)**2*(abs(cuentitas.a1VVF1(k0,thi,phi,th,phs,ep1,ep2,d))**2*cuentitas.w(s1,l1,k1,k2)+\
            abs(cuentitas.a1VVF2(k0,thi,phi,th,phs,ep1,ep2,d))**2*cuentitas.w(s2,l2,k1,k2)+\
            2*real(cuentitas.a1VVF1(k0,thi,phi,th,phs,ep1,ep2,d)*conj(cuentitas.a1VVF2(k0,thi,phi,th,phs,ep1,ep2,d)))*cuentitas.w_f1f2(s1,l1,s2,l2,k1,k2))

In [3]:
def sigma_vvSSA(ki,thi,phi,th,phs,ep0,ep1,ep2,d,s1,l1,s2,l2):
    
    k1 = ki*(sin(th)*cos(phs)-sin(thi)*cos(phi))   
    k2 = ki*(sin(th)*sin(phs)-sin(thi)*sin(phi))
    
    ksx = kx(ki,th,phs)
    ksy = ky(ki,th,phs)
    
    kix = kx(ki,thi,phi)
    kiy = ky(ki,thi,phi)
    
    return ki**4*np.cos(th)**2*np.cos(thi)/(2*np.pi)**2*(
        np.abs(X1_u(ksx,ksy,kix,kiy,ki,ep0,ep1,ep2,d)[0,0])**2*W(ksx-kix,ksy-kiy,s1,l1)+\
        np.abs(X1_b(ksx,ksy,kix,kiy,ki,ep0,ep1,ep2,d)[0,0])**2*W(ksx-kix,ksy-kiy,s2,l2))

In [4]:
def toFitSSA(ep0,ep1,ep2,d,s1,l1,s2,l2,angulo):
    landa = 0.025
    ki = 2*np.pi/landa
    phi = np.pi
    
    ### s0
    phs = phi + np.pi
    thi = angulo  #cambiar segun la zona
    thi = thi*np.pi/180 #T008, T061, T021
    ths = thi
    
    ksx = kx(ki,ths,phs)
    ksy = ky(ki,ths,phs)
    
    kix = kx(ki,thi,phi)
    kiy = ky(ki,thi,phi)
    
    aux = ki**4*np.cos(ths)**2*np.cos(thi)/(2*np.pi)**2*(
        np.abs(X1_u(ksx,ksy,kix,kiy,ki,ep0,ep1,ep2,d)[0,0])**2*W(ksx-kix,ksy-kiy,s1,l1)+\
        np.abs(X1_b(ksx,ksy,kix,kiy,ki,ep0,ep1,ep2,d)[0,0])**2*W(ksx-kix,ksy-kiy,s2,l2))
    s0s = 10*np.log10(aux)
    
    ### emi
    thi = 0.0001*np.pi/180
    
    ths_ = (X+1)*np.pi/4
    phs_ = (Y+1)*np.pi
    
    ths_1 = np.reshape(ths_, (1,m**2))
    phs_1 = np.reshape(phs_, (1,m**2))
    
    q = (np.sin(ths_1)*sigma_vvSSA(ki,thi,phi,ths_1,phs_1,ep0,ep1,ep2,d,s1,l1,s2,l2))
    aux = Wt*q
    I_gauss = (np.pi**2/4)*aux.sum()
 
    emi = 1 - (I_gauss/(4*np.pi))
 
    a = TT.zeros(4)
    aa = TT.set_subtensor(a[0], s0s[0])
    aaa = TT.set_subtensor(aa[1], s0s[1])
    aaaa = TT.set_subtensor(aaa[2], s0s[2])
    aaaaa = TT.set_subtensor(aaaa[3], emi)
 
    return aaaaa

In [4]:
def toFit(ep1,ep2,d,s1,l1,s2,l2,angulo):
    landa = 0.025
    k0 = 2*np.pi/landa
    phi = np.pi
    ### s0
    phs = phi + np.pi
    thi = angulo  #cambiar segun la zona
    thi = thi*np.pi/180 #DU2 T008, T061, T021
    ths = thi
    k1 = k0*(sin(ths)*cos(phs)-sin(thi)*cos(phi))
    k2 = k0*(sin(ths)*sin(phs)-sin(thi)*sin(phi))
 
    aux = 4*np.pi*k0**2*cos(ths)**2*(abs(cuentitas.a1VVF1(k0,thi,phi,ths,phs,ep1,ep2,d))**2*cuentitas.w(s1,l1,k1,k2)+\
            abs(cuentitas.a1VVF2(k0,thi,phi,ths,phs,ep1,ep2,d))**2*cuentitas.w(s2,l2,k1,k2)+\
            2*real(cuentitas.a1VVF1(k0,thi,phi,ths,phs,ep1,ep2,d)*conj(cuentitas.a1VVF2(k0,thi,phi,ths,phs,ep1,ep2,d)))*cuentitas.w_f1f2(s1,l1,s2,l2,k1,k2))
    s0s = 10*np.log10(aux)
    
    ### emi
    thi = 0.0001*np.pi/180
    
    ths_ = (X+1)*np.pi/4
    phs_ = (Y+1)*np.pi
    
    ths_1 = np.reshape(ths_, (1,m**2))
    phs_1 = np.reshape(phs_, (1,m**2))
    
    q = (np.sin(ths_1)*S0VV_Fran_completo(k0,thi,phi,ths_1,phs_1,ep1,ep2,d,s1,l1,s2,l2))
    aux = Wt*q
    I_gauss = (np.pi**2/4)*aux.sum()
 
    emi = 1 - (I_gauss/(4*np.pi))
 
    a = TT.zeros(4)
    aa = TT.set_subtensor(a[0], s0s[0])
    aaa = TT.set_subtensor(aa[1], s0s[1])
    aaaa = TT.set_subtensor(aaa[2], s0s[2])
    aaaaa = TT.set_subtensor(aaaa[3], emi)
 
    return aaaaa

In [5]:
def medicion(zona):     #angulo - observacion - nombre
    if zona == 'VP1':
        out = [np.asarray([20.44, 18.79, 17.36]),np.asarray([-6.13, -5.71, -5.66, 0.927]),'VP1']
    elif zona == 'VP2':
        out = [np.asarray([18.63, 17.19, 11.25]),np.asarray([-6.65, -6.17, -5.93, 0.937]),'VP2']
    elif zona == 'HU0':
        out = [np.asarray([24.59, 15.76, 20.69]),np.asarray([-4.07, -3.27, -3.59, 0.917]),'HU0']
    elif zona == 'HU1':
        out = [np.asarray([22.46, 15.33, 15.57]),np.asarray([-3.64, -3.12, -2.90, 0.921]),'HU1']
    elif zona == 'HU2':
        out = [np.asarray([18.86, 21.10, 12.98]),np.asarray([-4.08, -9.34, -3.55, 0.914]),'HU2']
    elif zona == 'DU1':
        out = [np.asarray([17.20, 20.10, 12.98]),np.asarray([-9.02, -9.55, -8.09, 0.943]),'DU1']
    elif zona == 'DU2':
        out = [np.asarray([23.84, 11.21, 12.64]),np.asarray([-11.96, -6.82, -7.99, 0.954]),'DU2']
    elif zona == 'DP1':
        out = [np.asarray([19.64, 17.76, 14.06]),np.asarray([-7.79, -8.34, -7.44, 0.947]),'DP1']
    elif zona == 'DP2':
        out = [np.asarray([24.21, 13.78, 16.76]),np.asarray([-11.25, -8.29, -8.62, 0.937]),'DP2']
    else:
      print('\n Inputs: VP1, VP2, HU0, HU1, HU2, DU1, DU2, DP1, DP2 \n')
    
    return out

In [6]:
def modeloSSA(sector, samples, sigmas):
 
    landa = 0.025
    k0 = 2*np.pi/landa
    phi = np.pi
 
    ranges = {}
    ranges['ep0'] = 1.4, 8
    ranges['ep1'] = 1.4, 8
    ranges['ep2'] = 1.4, 8
    ranges['d'] = 0.001, 0.1
    ranges['s1'] = 0.0004, 0.004
    ranges['l1'] = 0.005, 0.03
    ranges['s2'] = 0.0004, 0.004
    ranges['l2'] = 0.005, 0.03
 
    with pm.Model() as model:
        ep0 = pm.Uniform('ep0', ranges['ep0'][0], ranges['ep0'][1])
        ep1 = pm.Uniform('ep1', ranges['ep1'][0], ranges['ep1'][1])
        ep2 = pm.Uniform('ep2', ranges['ep2'][0], ranges['ep2'][1])
        d = pm.Uniform('d', ranges['d'][0], ranges['d'][1])
        s1 = pm.Uniform('s1', ranges['s1'][0], ranges['s1'][1])
        l1 = pm.Uniform('l1', ranges['l1'][0], ranges['l1'][1])
        s2 = pm.Uniform('s2', ranges['s2'][0], ranges['s2'][1])
        l2 = pm.Uniform('l2', ranges['l2'][0], ranges['l2'][1])
 
    def fSSA(ep0 = ep0, 
               ep1 = ep1,
               ep2 = ep2, 
               d = d, 
               s1 = s1, 
               l1 = l1,
               s2 = s2, 
               l2 = l2):
        return toFitSSA(ep0,ep1,ep2,d,s1,l1,s2,l2,sector[0])
 
    with model:
        function_pm = pm.Deterministic('s0f', fSSA())
        observations = pm.Normal( "obs",  mu=real(function_pm), sigma=sigmas, observed=sector[1])
        #trace = pm.sample_smc(samples, parallel=False)  #porque no corre las cadenas en parelelo? Es mas rapido False
        trace = pm.sample_smc(samples, cores=2, parallel=False) #probar con 4 
    
    return trace

In [11]:
def modelo(sector, samples, sigmas):
 
    landa = 0.025
    k0 = 2*np.pi/landa
    phi = np.pi
 
    ranges = {}
    ranges['ep1'] = 1.4, 8
    ranges['ep2'] = 1.4, 8
    ranges['d'] = 0.001, 0.1
    ranges['s1'] = 0.0004, 0.004
    ranges['l1'] = 0.005, 0.03
    ranges['s2'] = 0.0004, 0.004
    ranges['l2'] = 0.005, 0.03
 
    with pm.Model() as model:
        ep1 = pm.Uniform('ep1', ranges['ep1'][0], ranges['ep1'][1])
        ep2 = pm.Uniform('ep2', ranges['ep2'][0], ranges['ep2'][1])
        d = pm.Uniform('d', ranges['d'][0], ranges['d'][1])
        s1 = pm.Uniform('s1', ranges['s1'][0], ranges['s1'][1])
        l1 = pm.Uniform('l1', ranges['l1'][0], ranges['l1'][1])
        s2 = pm.Uniform('s2', ranges['s2'][0], ranges['s2'][1])
        l2 = pm.Uniform('l2', ranges['l2'][0], ranges['l2'][1])
 
    def f(ep1 = ep1, 
               ep2 = ep2, 
               d = d, 
               s1 = s1, 
               l1 = l1,
               s2 = s2, 
               l2 = l2):
        return toFit(ep1,ep2,d,s1,l1,s2,l2,sector[0])
 
    with model:
        function_pm = pm.Deterministic('s0f', f())
        observations = pm.Normal( "obs",  mu=real(function_pm), sigma=sigmas, observed=sector[1])
        #trace = pm.sample_smc(samples)  #porque no corre las cadenas en parelelo? Es mas rapido False
        trace = pm.sample_smc(samples, cores=2, parallel=False) #probar con 4 
    
    return trace

In [ ]:
zonaMed = 'HU2'
muestras = 125000
sigmas = np.asarray([0.1, 0.1, 0.1, 0.01])

traza = modelo(medicion(zonaMed),muestras,sigmas)


Sample initial stage: ...
Stage:   0 Beta: 0.000 Steps:  25 Acce: 1.000
Stage:   1 Beta: 0.002 Steps:  25 Acce: 0.226
Stage:   2 Beta: 0.008 Steps:  17 Acce: 0.121
Stage:   3 Beta: 0.022 Steps:  25 Acce: 0.071
Stage:   4 Beta: 0.047 Steps:  25 Acce: 0.044
Stage:   5 Beta: 0.067 Steps:  25 Acce: 0.030
Stage:   6 Beta: 0.083 Steps:  25 Acce: 0.025
Stage:   7 Beta: 0.102 Steps:  25 Acce: 0.027


In [ ]:
zonaMed = 'HU2'
muestras = 50000
sigmas = np.asarray([0.1, 0.1, 0.1, 0.01])

traza = modelo(medicion(zonaMed),muestras,sigmas)

In [ ]:
e1 = traza['ep1']
e2 = traza['ep2']
d = traza['d']
s1 = traza['s1']
l1 = traza['l1']
s2 = traza['s2']
l2 = traza['l2']

guardar = [e1,e2,d,s1,l1,s2,l2]
#ruta = '/content/drive/MyDrive/Tesis de Javi/DatosSimu/'+zonaMed+'-s1l1-s=50k.txt'
#ruta = '/content/drive/MyDrive/Tesis de Javi/DatosSimu/Nuevos/'+zonaMed+'-N='+str(n)+'-S='+str(muestras)+'.txt'
f = np.savetxt(zonaMed+'-N='+str(n)+'-S='+str(muestras)+'SSA.txt', guardar, delimiter=';')
print('Archivo guardado')


In [ ]:
#Ploteo
print('Ploteando...')

plotear = np.stack((traza['ep1'], traza['ep2'], traza['d'], traza['s1'], traza['l1'], traza['s2'], traza['l2'])).T 
dfplotear = pd.DataFrame(plotear, columns = ['ep1','ep2','d','s1','l1','s2','l2'])

g = sns.PairGrid(dfplotear)
g.fig.suptitle(zonaMed, y=1.01, fontsize = 'xx-large')
g.map_upper(sns.histplot)
g.map_lower(sns.kdeplot, fill=True)
g.map_diag(sns.histplot, kde=True, color='.5')
#g.savefig('prueba.png')
#files.download('prueba.png') #descarga la imagen



---
# Pruebas a segundo orden


In [ ]:
thInc = medicion('HU2')[0]
thSc = thInc
phInc = np.pi
phSc = phInc + np.pi

landa = 0.025
k0 = 2*np.pi/landa

l2 = 0.015
s2 = 0.00085
l1 = 0.015
s1 = 0.0014
d = 0.009
ep2 = 6.25

ep1 = np.linspace(1.4, 4.7, 1000)

import matplotlib.pyplot as plt




In [ ]:
plt.figure(figsize=[10.0,7.5])
for i in range(len(thInc)):
  plt.plot(ep1, cuentitas.a1VVF1(k0,thInc[i],phInc,thSc[i],phSc,ep1,ep2,d),label='thi: '+str(thInc[i]))


plt.legend()
plt.xlabel('ep1')
plt.ylabel('a1VV')
plt.title('HU2')

